In [1]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
# Load the dataset
df = pd.read_csv('C:/.../model_df.csv')

In [3]:
# Label encoding
label_encoder_success = LabelEncoder()
label_encoder_PSP = LabelEncoder()
label_encoder_3D_secured = LabelEncoder()
label_encoder_unique_transaction = LabelEncoder()

df['success_encoded'] = label_encoder_success.fit_transform(df['success'])
df['PSP_encoded'] = label_encoder_PSP.fit_transform(df['PSP'])
df['3D_secured_encoded'] = label_encoder_3D_secured.fit_transform(df['3D_secured'])
df['unique_transaction_encoded'] = label_encoder_unique_transaction.fit_transform(df['unique_transaction'])

In [4]:
# Weekday mapping and encoding
weekday_mapping = {
    "monday": 0,
    "tuesday": 1,
    "wednesday": 2,
    "thursday": 3,
    "friday": 4,
    "saturday": 5,
    "sunday": 6
}
df['weekday_encoded'] = df['weekday'].str.lower().map(weekday_mapping)

# Feature selection
X_new = df.loc[:, ['PSP_encoded', 'weekday_encoded', '3D_secured_encoded', 'amount']]
y = df['success_encoded']

# Split the dataset
X_new_train, X_new_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)

# Encoding mappings for PSP and 3D secured
psp_mapping = {
    "moneycard": 0,
    "goldcard": 1,
    "uk_card": 2,
    "simplecard": 3
}

secured_mapping = {
    "yes": 1,
    "no": 0
}

# Service fee calculation
def calculate_service_fee(psp, success):
    if psp == 'moneycard':
        return 5 if success == 'successful' else 2
    elif psp == 'goldcard':
        return 10 if success == 'successful' else 5
    elif psp == 'uk_card':
        return 3 if success == 'successful' else 1
    elif psp == 'simplecard':
        return 1 if success == 'successful' else 0.5
    else:
        return 0  # Default value for unexpected PSP

# Gradient Boosting Classifier fitting
gb = GradientBoostingClassifier(random_state=42, n_estimators=100, learning_rate=0.2, max_depth=3, subsample=1.0)
gb.fit(X_new_train, y_train)

# Prediction function
def predict_success_probability():
    try:
        psp = combo_psp.get().lower()
        psp_encoded = psp_mapping.get(psp)

        if psp_encoded is None:
            raise KeyError(f"PSP '{psp}' not recognized. Please check the input.")

        sec = secured_mapping[combo_3d.get().lower()]
        weekday = weekday_mapping[combo_weekday.get().lower()]
        amt = float(entry_amount.get())

        # Prepare input data as a DataFrame
        input_data = pd.DataFrame([[psp_encoded, weekday, sec, amt]], columns=['PSP_encoded', 'weekday_encoded', '3D_secured_encoded', 'amount'])
        
        # Predict success probability
        prob = gb.predict_proba(input_data)[0, 1]  # Probability of success (1)

        # Determine success status based on threshold
        success_status = 'successful' if prob >= 0.5 else 'unsuccessful'
        service_fee = calculate_service_fee(psp, success_status)

        # Display the result
        result_label.config(text=f"Success Probability: {prob:.4f}\n"
                                 f"Status: {success_status}\n"
                                 f"Service Fee: €{service_fee:.2f}")
    except ValueError as e:
        result_label.config(text=f"Value Error: {str(e)}. Please enter valid input values.")
    except KeyError as e:
        result_label.config(text=f"Key Error: {str(e)}")
    except Exception as e:
        result_label.config(text=f"Unexpected Error: {str(e)}")

In [7]:
# Create the main window
root = tk.Tk()
root.title("Success Probability Predictor")
root.geometry('400x300')  # Set the size of the window
root.configure(bg='#f2f2f2')  # Set background color

# Set the default font style for widgets
root.option_add('*Font', 'Helvetica 12')

# Create and place the PSP dropdown (combobox)
ttk.Label(root, text="PSP:", background='#f2f2f2').grid(column=0, row=0, padx=20, pady=10, sticky='E')
combo_psp = ttk.Combobox(root, values=list(psp_mapping.keys()), state="readonly", width=18)
combo_psp.grid(column=1, row=0, padx=20, pady=10, sticky='W')

# Create and place the 3D secured dropdown (combobox)
ttk.Label(root, text="3D Secured:", background='#f2f2f2').grid(column=0, row=1, padx=20, pady=10, sticky='E')
combo_3d = ttk.Combobox(root, values=list(secured_mapping.keys()), state="readonly", width=18)
combo_3d.grid(column=1, row=1, padx=20, pady=10, sticky='W')

# Create and place the Weekday dropdown (combobox)
ttk.Label(root, text="Weekday:", background='#f2f2f2').grid(column=0, row=2, padx=20, pady=10, sticky='E')
combo_weekday = ttk.Combobox(root, values=list(weekday_mapping.keys()), state="readonly", width=18)
combo_weekday.grid(column=1, row=2, padx=20, pady=10, sticky='W')

# Create and place the Amount entry
ttk.Label(root, text="Amount (€):", background='#f2f2f2').grid(column=0, row=3, padx=20, pady=10, sticky='E')
entry_amount = ttk.Entry(root, width=20)
entry_amount.grid(column=1, row=3, padx=20, pady=10, sticky='W')

# Create and place the Predict button
predict_button = ttk.Button(root, text="Predict", command=predict_success_probability, width=15)
predict_button.grid(column=0, row=4, columnspan=2, pady=20)

# Create and place the result label
result_label = ttk.Label(root, text="Enter values and click 'Predict'", background='#f2f2f2', anchor='center', wraplength=300)
result_label.grid(column=0, row=5, columnspan=2, padx=20, pady=10)

# Run the application
root.mainloop()